In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img

In [ ]:
!pip install split-folders

In [ ]:
import kagglehub
kagglehub.dataset_download("ryanbadai/clothes-dataset")

In [ ]:
import splitfolders
splitfolders.ratio("/kaggle/input/clothes-dataset/Clothes_Dataset", output="/content/clothes", ratio=(0.6, 0.2, 0.2), group_prefix=None)

In [ ]:
path = "/content/clothes/train/Hoodie/halfzip_hoodie_awk_1697034196_0d90db76_progressive_thumbnail.jpg"
img = load_img(path, target_size=(299, 299))

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras import layers, models

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_datagen.flow_from_directory(
    "/content/clothes/train",
    target_size=(299, 299),
    batch_size=32)

In [ ]:
train_data.class_indices

In [ ]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_data = val_datagen.flow_from_directory(
    "/content/clothes/val",
    target_size=(299, 299),
    batch_size=32,
    shuffle=False
)

In [ ]:
base_model = Xception(weights="imagenet", input_shape=(150, 150, 3), include_top=False)

base_model.trainable = False

In [ ]:
model = models.Sequential([
    base_model,

    layers.GlobalAveragePooling2D(),
    layers.Dense(15)
])

In [ ]:
optimizer = keras.optimizers.Adam()
loss = keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(train_data, epochs=10, validation_data=val_data)

In [ ]:
def make_model(learning_rate):
  base_model = Xception(weights="imagenet", input_shape=(150, 150, 3), include_top=False)

  base_model.trainable = False

  model = models.Sequential([
    base_model,

    layers.GlobalAveragePooling2D(),
    layers.Dense(15)
  ])

  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  loss = keras.losses.CategoricalCrossentropy(from_logits=True)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  return model

In [ ]:
scores = {}

for lr in [0.0001, 0.001, 0.01, 0.1]:
    print(lr)

    model = make_model(learning_rate=lr)
    history = model.fit(train_data, epochs=10, validation_data=val_data)
    scores[lr] = history.history

    print()
    print()

In [ ]:
def make_model(learning_rate, inner_size):
  base_model = Xception(weights="imagenet", input_shape=(150, 150, 3), include_top=False)

  base_model.trainable = False

  model = models.Sequential([
    base_model,

    layers.GlobalAveragePooling2D(),
    layers.Dense(inner_size, activation='relu'),
    layers.Dense(15)
  ])

  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  loss = keras.losses.CategoricalCrossentropy(from_logits=True)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  return model

In [ ]:
learning_rate = 0.001

scores = {}

for size in [20, 50, 100, 1000]:
    print(size)

    model = make_model(learning_rate=learning_rate, inner_size=size)
    history = model.fit(train_data, epochs=10, validation_data=val_data)
    scores[size] = history.history

    print()
    print()

In [ ]:
def make_model(learning_rate, inner_size, droprate):
  base_model = Xception(weights="imagenet", input_shape=(299, 299, 3), include_top=False)

  base_model.trainable = False

  model = models.Sequential([
    base_model,

    layers.GlobalAveragePooling2D(),
    layers.Dense(inner_size, activation='relu'),
    layers.Dropout(droprate),
    layers.Dense(15)
  ])

  optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
  loss = keras.losses.CategoricalCrossentropy(from_logits=True)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  return model

In [ ]:
learning_rate = 0.001
size = 100

scores = {}

for droprate in [0.2, 0.5, 0.8]:
    print(droprate)

    model = make_model(
        learning_rate=learning_rate,
        inner_size=size,
        droprate=droprate
    )

    history = model.fit(train_data, epochs=10, validation_data=val_data)
    scores[droprate] = history.history

    print()
    print()

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

train_data = train_datagen.flow_from_directory(
    "/content/clothes/train",
    target_size=(299, 299),
    batch_size=32)

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
model = make_model(learning_rate=0.001, inner_size=100, droprate=0.2)
model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[checkpoint])

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_data = test_datagen.flow_from_directory(
    "/content/clothes/test",
    target_size=(299, 299),
    batch_size=32,
    shuffle=False
)

In [ ]:
model = keras.models.load_model('/content/xception_06_0.720.h5')

In [ ]:
model.evaluate(test_data)

In [ ]:
path = "/content/clothes/test/Sweter/060062e8-b564-4aaa-8d9e-18e735915e37.jpg"

img = load_img(path, target_size=(299, 299))

In [ ]:
X = np.array([img])

X.shape

In [ ]:
X = preprocess_input(X)

In [ ]:
predictions = model.predict(X)

In [ ]:
classes = [
  'Blazer',
  'Celana_Panjang',
  'Celana_Pendek',
  'Gaun',
  'Hoodie',
  'Jaket',
  'Jaket_Denim',
  'Jaket_Olahraga',
  'Jeans',
  'Kaos',
  'Kemeja',
  'Mantel',
  'Polo',
  'Rok',
  'Sweter'
]

In [ ]:
scores = predictions[0]

results = [(classes[i], float(score)) for i, score in enumerate(scores)]

results.sort(key=lambda x: x[1], reverse=True)

output = {name: score for name, score in results}

In [ ]:
output